In [1]:
# !pip uninstall cello_multiplier --yes
# !pip install -U git+https://github.com/Bishop-Laboratory/CellO-MultiPLIER.git@test

In [2]:
import sys
sys.path.append('../../../CellO-MultiPLIER/src')

In [3]:
import cello_multiplier as cm
from onto_lib import general_ontology_tools as got
import pandas as pd

Unable to import pygraphviz. Visualization is disabled.
Loading ontology from ../../../CellO-MultiPLIER/src\onto_lib\obo\DOID.17-01-30.obo ...
Loading ontology from ../../../CellO-MultiPLIER/src\onto_lib\obo\UBERON.17-01-30.obo ...
Loading ontology from ../../../CellO-MultiPLIER/src\onto_lib\obo\CL.18-11-13.obo ...
Loading ontology from ../../../CellO-MultiPLIER/src\onto_lib\obo\CVCL.17-01-30.obo ...
Loading ontology from ../../../CellO-MultiPLIER/src\onto_lib\obo\EFO.17-01-30.obo ...


In [4]:
B_df, Z_df, labels, per_gene_mean, per_gene_std = cm.get_default_mats()

In [5]:
del B_df, Z_df

In [6]:
#split 
from pathlib import Path
import json
import numpy as np

data_dir = Path('../../data')
regr_data_root = data_dir / 'experiments' / 'positive_b-split_256'

In [7]:
cello_dir = data_dir / 'CellO_data/bulk_RNA_seq_training_set'
split_dir = cello_dir / 'pretraining_validation_split'

with open(split_dir / 'validation_bulk_experiments.json', 'r') as f:
    validation_egs = json.load(f)

with open(split_dir / 'pre_training_bulk_experiments.json', 'r') as f:
    train_egs = json.load(f)

In [8]:
pos_b = np.load(regr_data_root / 'b_pos.npy')

In [9]:
samples = np.load(regr_data_root / 'samples.npy', allow_pickle=True)
# test_samples = np.load(regr_data_root / 'test_samples.npy', allow_pickle=True)
# all_samples = pd.Series(np.concatenate([train_samples, test_samples]))

In [10]:
B_df = pd.DataFrame(pos_b, index=samples)
B_df.shape

(4293, 256)

In [11]:
train_B_df = B_df[B_df.index.isin(train_egs)]
train_B_df.shape

(3609, 256)

In [12]:
test_B_df = B_df[B_df.index.isin(validation_egs)]
test_B_df.shape

(684, 256)

In [13]:
# create list of samples by celltypes
from sklearn.preprocessing import MultiLabelBinarizer

sample2types = {
            sample: list(map(got.get_term_name, types_ids))
            for sample, types_ids in labels.items()
        }

types_per_b_samples = B_df.index.map(sample2types).values

mlb = MultiLabelBinarizer()

samples_dummies = pd.DataFrame(mlb.fit_transform(types_per_b_samples), columns=mlb.classes_, index=B_df.index)

celltypes = samples_dummies.columns.tolist()

type2samples = {
    type_: samples_dummies.index[samples_dummies[type_] == 1].tolist()
    for type_ in celltypes
}

In [14]:
# create target list for given cell type with 1 being that cell type and 0 being any other cell type
def set_target(celltype, type2samples, train_Y_df, test_Y_df, oversample, max_neg_pos_ratio):
    samplelist = type2samples[celltype]
    samplelist_train = [x for x in samplelist if x in train_Y_df.index.values]
    samplelist_test = [x for x in samplelist if x in test_Y_df.index.values]
    
    if(oversample == True and len(samplelist_train) > 0 and len(samplelist_test) >0):
        len_negative_train = len(train_Y_df)-len(samplelist_train)
        len_positive_train = len(samplelist_train)
        
        neg_pos_ratio = len_negative_train / len_positive_train
        
        # limit the max oversampling ratio
        neg_pos_ratio = min(neg_pos_ratio, max_neg_pos_ratio)
        
        if(neg_pos_ratio > 1):
            df_train = train_Y_df.loc[samplelist_train].sample(
                    n=int((neg_pos_ratio - 1) * len_positive_train),
                    replace=True, random_state=111
                )
            train_Y_df = train_Y_df.append(df_train)
    
    target_train = pd.Series(0, index = train_Y_df.index)
    target_train.loc[samplelist_train] = 1
    
    target_test = pd.Series(0, index = test_Y_df.index)
    target_test.loc[samplelist_test] = 1
    
#     train_Y_transformed = scaler.transform(train_Y_df)
#     test_Y_transformed = scaler.transform(test_Y_df)
    
    return(target_train.values, target_test.values, train_Y_df, test_Y_df )
#     return(target_train.values, target_test.values, train_Y_df, test_Y_df)

In [15]:
def fscore(p, r):
    denom = p + r or 1

    return (p * r) / denom

In [16]:
# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score
import numpy as np
import warnings
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [18]:
types_sizes = samples_dummies.sum()
types_with_data = types_sizes[types_sizes > 50].index.values
all_types = samples_dummies.columns

In [19]:
warnings.filterwarnings('ignore') #gets rid of sklearn convergence warning

successful_celltypes = []
unsuccessful_celltypes = []
test_prs = []
test_precision = []
test_recall = []
fscores = []

for cell_type in tqdm(types_with_data):
    train_target, train_test, train_data, test_data = set_target(
        cell_type, type2samples, train_B_df, test_B_df, oversample=True,
        max_neg_pos_ratio=2
    )
    
    if (1 in train_target and 1 in train_test and 0 in train_target and 0 in train_test):
        # lasso penalty
        clf = LogisticRegression(solver = "saga",penalty = "l1",random_state=111 )
        
        clf.fit(train_data, train_target)
        target_pred = clf.predict(test_data)
        test_pr = metrics.average_precision_score(train_test, target_pred)
        test_precision += [precision_score(train_test, target_pred)]
        test_recall += [recall_score(train_test, target_pred)]
        fscores += [fscore(test_precision[-1], test_recall[-1])]
        successful_celltypes += [cell_type]
    else:
        unsuccessful_celltypes += [cell_type]
        
p = np.mean(test_precision)
r = np.mean(test_recall)
f = np.mean(fscores)
f_micro = fscore(p, r) 

report = pd.DataFrame(list(zip(successful_celltypes,test_precision,test_recall, fscores)),
                      columns = ["celltype","precision score","recall score","f score"])


  0%|          | 0/106 [00:00<?, ?it/s]

In [30]:
print(f'precision: {p:.4f}, recall: {r:.4f}, f1: {f:.4f}, f1 micro avg: {f_micro:.4f}')

precision: 0.8854, recall: 0.6771, f1: 0.3541, f1 micro avg: 0.3837


In [20]:
report = pd.DataFrame(
    {
        "celltype": successful_celltypes,
        "precision score": test_precision,
        "recall score": test_recall,
        "f score": fscores
    })

## KNN classifier

In [23]:
B_train_mat = train_B_df.values
B_train_mat.shape

(3609, 256)

In [24]:
b_test_mat = test_B_df.values

In [25]:
import scipy

def get_pearson_dists(vectors_a, vectors_b):
    vectors_pairs = zip(vectors_a, vectors_b)
    return [scipy.spatial.distance.correlation(a, b) for a, b in vectors_pairs]

def get_pearson_dists_mat(vectors_a, vectors_b):
    return np.array([
        # repeat the A vector for each B vector
        get_pearson_dists(np.array(vector_a[np.newaxis, :]).repeat(len(vectors_b), axis=0), vectors_b)
        for vector_a in tqdm(vectors_a)
    ])

### Predict all cell types for all test samples

In [42]:
test_2_train_dists = scipy.spatial.distance.cdist(B_train_mat, b_test_mat, metric='correlation')
test_2_train_dists.shape

(3609, 684)

In [43]:
train_samples, test_samples = train_B_df.index.values, test_B_df.index.values

In [44]:
# distances is n_train matrix X n_test
# with columns for test items and rows for train items
dists_df = pd.DataFrame(test_2_train_dists)
dists_df.columns = test_samples
dists_df.index = train_samples

In [45]:
def knn_classify(n_neighbours):
    # map each test item to it's closest train ones using dists matrix
    test_sample2closest_train = {
        # get the column for current test sample and find the index of raw the closest value
        sample_id: dists_df[sample_id].sort_values().index[:n_neighbours]
        for sample_id in test_samples
    }
    
    # predict each cell type just by looking N closest items for each test sample
    cell_types_y_predicted = pd.DataFrame([
        samples_dummies.loc[test_sample2closest_train[test_id]].sum()
        for test_id in test_samples
    ])
    
    proba = cell_types_y_predicted.values / n_neighbours

    cell_types_y_predicted[:] = proba
    
    return cell_types_y_predicted

### Calculate performance stats

In [46]:
true_test_dummies = samples_dummies.loc[test_samples]
true_test_dummies.shape

(684, 317)

In [47]:
warnings.filterwarnings('ignore') #gets rid of sklearn convergence warning

all_scores = []
for n_neighbours in tqdm([1, 5, 10, 15, 20, 30, 40, 50, 100]):
    cell_types_y_predicted_proba = knn_classify(n_neighbours)
    
    for th in [0.1, .3, .5, .7, .9]:
        cell_types_y_predicted = cell_types_y_predicted_proba.copy()
        above_threshold = cell_types_y_predicted.values > th

        cell_types_y_predicted.values[above_threshold] = 1
        cell_types_y_predicted.values[~above_threshold] = 0

        successful_celltypes = []
        unsuccessful_celltypes = []
        test_precision = []
        test_recall = []
        f_scores = []

        # types_with_data or all_types
        for cell_type in all_types: 
            # cell type information from original data
            y_true = true_test_dummies[cell_type]
            y_predicted = cell_types_y_predicted[cell_type]

            # calculate precision only if we have predictions for this cell type
            if y_predicted.sum() > 0:
                test_precision.append(precision_score(y_true, y_predicted))

            # calculate recall only if we have true activations for this cell type
            if y_true.sum() > 0:
                test_recall.append(recall_score(y_true, y_predicted))

            # calculate f1 score only if we have precision and recall
            if y_predicted.sum() > 0 and y_true.sum() > 0:
                f_scores.append(fscore(test_precision[-1], test_recall[-1]))

        p = np.mean(test_precision)
        r = np.mean(test_recall)
        f = np.mean(f_scores)

        f_micro = fscore(p, r)
        
        all_scores.append({
            'knn': n_neighbours,
            'threshold': th,
            'p': p,
            'r': r,
            'f': f,
            'f_micro': f_micro
        })

#         print(f'knn: {n_neighbours}, th: {th}, precision: {p:.4f}, recall: {r:.4f}, f1: {f:.4f}, f1 micro avg: {f_micro:.4f}')
        
        if n_neighbours == 1:
            break

  0%|          | 0/9 [00:00<?, ?it/s]

In [48]:
pd.DataFrame(all_scores).sort_values('f_micro', ascending=False).head(10)

,knn,threshold,p,r,f,f_micro
0,1,0.1,0.679225,0.586178,0.341586,0.314640
2,5,0.3,0.610486,0.622493,0.328983,0.308215
3,5,0.5,0.725400,0.529604,0.337805,0.306114
6,10,0.1,0.531758,0.662645,0.317015,0.295015
11,15,0.1,0.495675,0.701542,0.308593,0.290454
16,20,0.1,0.522346,0.642271,0.309210,0.288067
1,5,0.1,0.484200,0.666710,0.312847,0.280492
26,40,0.1,0.542163,0.577830,0.301117,0.279714
7,10,0.3,0.619745,0.504777,0.326698,0.278192
21,30,0.1,0.516767,0.599267,0.301421,0.277484
